In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
stock = ['SH600000','SH600010','SH600015','SH600016','SH600018','SH600028','SH600030','SH600036','SH600048','SH600050','SH600089','SH600104','SH600109','SH600111','SH600150','SH600256','SH600406','SH600518','SH600519','SH600583','SH600585','SH600637','SH600690','SH600837','SH600887','SH600893','SH600958','SH600999','SH601006','SH601088','SH601166','SH601169','SH601186','SH601288','SH601318','SH601328','SH601390','SH601398','SH601601','SH601628','SH601668','SH601688','SH601766','SH601800','SH601818','SH601857','SH601901','SH601988','SH601989','SH601998']

In [3]:
folder_name = "raw data 17-19"
dirs = os.listdir(folder_name)

close = {}

for file in dirs:
    if file.endswith(".csv"):
        close[str(file).replace('.csv','')] = pd.read_csv(folder_name+'/'+file,index_col = 0)
    else:
        continue

In [4]:
for i in stock:
    close[i] = close[i].iloc[504:]
    close[i].reset_index(inplace = True)
    close[i].rename(columns={"Time": "day"}, inplace = True)
    close[i] = close[i][['day','Close']]
    close[i]['day'] = np.array(range(505,757))
    

In [5]:
close['SH600000']

,day,Close
0,505,131.0422
1,506,133.0164
2,507,130.4252
3,508,130.0550
4,509,126.4767
...,...,...
247,752,158.0162
248,753,155.2172
249,754,156.2351
250,755,155.7262


In [6]:
folder_name = "label 5"
dirs = os.listdir(folder_name)

label5 = {}

for file in dirs:
    if file.endswith(".csv"):
        label5[str(file).replace('.csv','')] = pd.read_csv(folder_name+'/'+file,index_col = 0)
    else:
        continue

In [7]:
for i in stock:
    label5[i] = label5[i].iloc[1:]
    label5[i].reset_index(inplace = True)
    label5[i].rename(columns={"index": "day", "preds":"label5_preds"}, inplace = True)
    label5[i] = label5[i][['day','label5_preds']]
    label5[i].loc[len(label5[i])] = [756, 0]

In [8]:
label5['SH600000']

,day,label5_preds
0,505,0.0
1,506,0.0
2,507,0.0
3,508,0.0
4,509,1.0
...,...,...
247,752,0.0
248,753,0.0
249,754,0.0
250,755,0.0


In [9]:
# save all raw data into a dictionary and use the stock name as the key of the dict
folder_name = "label 10"
dirs = os.listdir(folder_name)

label10 = {}

for file in dirs:
    if file.endswith(".csv"):
        label10[str(file).replace('.csv','')] = pd.read_csv(folder_name+'/'+file,index_col = 0)
    else:
        continue

In [10]:
for i in stock:
    label10[i] = label10[i].iloc[1:]
    label10[i].reset_index(inplace = True)
    label10[i].rename(columns={"index": "day","preds":"label10_pred"}, inplace = True)
    label10[i] = label10[i][['day','label10_pred']]
    label10[i].loc[len(label10[i])] = [756, 0]

In [11]:
label10['SH600000']

,day,label10_pred
0,505,0.0
1,506,0.0
2,507,0.0
3,508,1.0
4,509,1.0
...,...,...
247,752,0.0
248,753,0.0
249,754,1.0
250,755,1.0


In [12]:
# save all raw data into a dictionary and use the stock name as the key of the dict
folder_name = "regression output"
dirs = os.listdir(folder_name)

lr = {}

for file in dirs:
    if file.endswith(".csv"):
        lr[str(file).replace('.csv','')] = pd.read_csv(folder_name+'/'+file,index_col = 0)
    else:
        continue

In [13]:
for i in stock:    
    lr[i].reset_index(inplace = True)
    lr[i].rename(columns={"index": "day","Predict":"LR_pred"}, inplace = True)
    lr[i] = lr[i][['day','LR_pred']]
    lr[i]['day'] = np.array(range(505,757))
    

In [14]:
lr['SH600000']

,day,LR_pred
0,505,0.016632
1,506,0.016826
2,507,0.016473
3,508,0.015751
4,509,0.015929
...,...,...
247,752,0.001747
248,753,0.002300
249,754,0.002846
250,755,0.003374


### Merging all into one

In [15]:
for i in stock:
    close[i] = close[i].merge(label5[i], left_on='day', right_on='day', how='inner')
    close[i] = close[i].merge(label10[i], left_on='day', right_on='day', how='inner')
    close[i] = close[i].merge(lr[i], left_on='day', right_on='day', how='inner')
    

In [16]:
close['SH600000']

,day,Close,label5_preds,label10_pred,LR_pred
0,505,131.0422,0.0,0.0,0.016632
1,506,133.0164,0.0,0.0,0.016826
2,507,130.4252,0.0,0.0,0.016473
3,508,130.0550,0.0,1.0,0.015751
4,509,126.4767,1.0,1.0,0.015929
...,...,...,...,...,...
247,752,158.0162,0.0,0.0,0.001747
248,753,155.2172,0.0,0.0,0.002300
249,754,156.2351,0.0,1.0,0.002846
250,755,155.7262,0.0,1.0,0.003374


# Running the simulation (Day 505 - Day 755)

In [17]:
stock_owned = dict((x,0) for x in stock) 
cash_owned = dict((x,1000000/50) for x in stock)



for x in stock:
    close[x]['stock owned'] = "" #close[x].iloc[i,5]
    close[x]['cash owned'] = "" #close[x].iloc[i,6]
    close[x]['total asset'] = "" #close[x].iloc[i,7]
    
    
    for i in range(0,len(close[x])-1): 
        price = close[x].iloc[i,1]
        buy_share = int(cash_owned[x]/price)
        
        if close[x].iloc[i,2] == 1 and close[x].iloc[i,3] == 1 and close[x].iloc[i,4] >= 0.05:  
            if cash_owned[x] >= buy_share * (price * (1 + 0.00065)):
                cash_owned[x] = round(cash_owned[x] - (buy_share * (price * (1 + 0.00065))),2)
                stock_owned[x]+=buy_share

            else:
                pass
        
        elif (close[x].iloc[i,2] == 0 and close[x].iloc[i,3] == 0) or close[x].iloc[i,4] <= 0:
            if stock_owned[x] >= 1:
                sell_share = stock_owned[x] 
                cash_owned[x] = round(cash_owned[x] + (sell_share * (price * (1 - 0.00065))),2)
                stock_owned[x]-= sell_share
                    
        close[x].iloc[i,5] = stock_owned[x]
        close[x].iloc[i,6] = cash_owned[x]
        close[x].iloc[i,7] = cash_owned[x] + round(stock_owned[x]*(price* (1 - 0.00065)),2)

# Selling Everything on Day 756

In [18]:
# LAST DAY
for x in stock:
    sell_share = stock_owned[x]
    
    cash_owned[x] = round(cash_owned[x] + (sell_share * (price * (1 - 0.00065))),2)
    stock_owned[x]-= sell_share
    
    close[x].iloc[len(close[x])-1,5] = stock_owned[x]
    close[x].iloc[len(close[x])-1,6] = cash_owned[x]
    close[x].iloc[len(close[x])-1,7] = cash_owned[x] + stock_owned[x]*price

In [19]:
sum(stock_owned.values())

0

# Metric 1: Amount of money at the end of Day 756

In [20]:
final_amt = sum(cash_owned.values())
print(f'${final_amt} is collected by the end of Day 756.')

$1055430.27 is collected by the end of Day 756.


In [21]:
perc_return = (final_amt - 1000000)/1000000
perc_return

0.05543027000000002

In [22]:
cash_owned

{'SH600000': 20000.0,
 'SH600010': 19340.49,
 'SH600015': 20000.0,
 'SH600016': 20000.0,
 'SH600018': 20000.0,
 'SH600028': 22446.25,
 'SH600030': 33193.43,
 'SH600036': 20000.0,
 'SH600048': 20000.0,
 'SH600050': 20000.0,
 'SH600089': 20000.0,
 'SH600104': 20000.0,
 'SH600109': 33095.47,
 'SH600111': 20000.0,
 'SH600150': 5952.78,
 'SH600256': 20000.0,
 'SH600406': 20000.0,
 'SH600518': 20000.0,
 'SH600519': 24697.61,
 'SH600583': 20000.0,
 'SH600585': 20000.0,
 'SH600637': 26611.47,
 'SH600690': 20000.0,
 'SH600837': 20000.0,
 'SH600887': 20000.0,
 'SH600893': 20000.0,
 'SH600958': 20000.0,
 'SH600999': 33158.9,
 'SH601006': 20401.48,
 'SH601088': 23613.05,
 'SH601166': 22805.58,
 'SH601169': 20000.0,
 'SH601186': 20000.0,
 'SH601288': 20000.0,
 'SH601318': 20000.0,
 'SH601328': 20000.0,
 'SH601390': 20000.0,
 'SH601398': 20000.0,
 'SH601601': 4877.35,
 'SH601628': 34772.61,
 'SH601668': 20000.0,
 'SH601688': 28690.66,
 'SH601766': 20000.0,
 'SH601800': 21773.14,
 'SH601818': 20000.0

In [23]:
stock_owned

{'SH600000': 0,
 'SH600010': 0,
 'SH600015': 0,
 'SH600016': 0,
 'SH600018': 0,
 'SH600028': 0,
 'SH600030': 0,
 'SH600036': 0,
 'SH600048': 0,
 'SH600050': 0,
 'SH600089': 0,
 'SH600104': 0,
 'SH600109': 0,
 'SH600111': 0,
 'SH600150': 0,
 'SH600256': 0,
 'SH600406': 0,
 'SH600518': 0,
 'SH600519': 0,
 'SH600583': 0,
 'SH600585': 0,
 'SH600637': 0,
 'SH600690': 0,
 'SH600837': 0,
 'SH600887': 0,
 'SH600893': 0,
 'SH600958': 0,
 'SH600999': 0,
 'SH601006': 0,
 'SH601088': 0,
 'SH601166': 0,
 'SH601169': 0,
 'SH601186': 0,
 'SH601288': 0,
 'SH601318': 0,
 'SH601328': 0,
 'SH601390': 0,
 'SH601398': 0,
 'SH601601': 0,
 'SH601628': 0,
 'SH601668': 0,
 'SH601688': 0,
 'SH601766': 0,
 'SH601800': 0,
 'SH601818': 0,
 'SH601857': 0,
 'SH601901': 0,
 'SH601988': 0,
 'SH601989': 0,
 'SH601998': 0}

In [24]:
close['SH600150']

,day,Close,label5_preds,label10_pred,LR_pred,stock owned,cash owned,total asset
0,505,43.5061,1.0,1.0,0.070575,459,17.72,19974
1,506,43.9983,0.0,1.0,0.070350,459,17.72,20199.8
2,507,44.8185,0.0,0.0,0.069974,0,20576,20576
3,508,44.5889,0.0,0.0,0.069574,0,20576,20576
4,509,44.0311,0.0,1.0,0.070049,0,20576,20576
...,...,...,...,...,...,...,...,...
247,752,71.0852,1.0,1.0,0.053183,0,47577.2,47577.2
248,753,70.1616,1.0,1.0,0.053001,0,47577.2,47577.2
249,754,71.4151,1.0,1.0,0.053152,0,47577.2,47577.2
250,755,72.0088,1.0,1.0,0.053664,660,20.53,47515.4


# Metric 2: Sharpe Ratio

In [25]:
# Calculating PNL
for x in stock:
    close[x]['pnl'] = "" #close[x].iloc[i,8]
    close[x].iloc[0,8] = 0
    
    for i in range(1,len(close[x])):
        close[x].iloc[i,8] = (close[x].iloc[i,7]/(close[x].iloc[i-1,7]))-1

In [26]:
close['SH600150']

,day,Close,label5_preds,label10_pred,LR_pred,stock owned,cash owned,total asset,pnl
0,505,43.5061,1.0,1.0,0.070575,459,17.72,19974,0
1,506,43.9983,0.0,1.0,0.070350,459,17.72,20199.8,0.0113032
2,507,44.8185,0.0,0.0,0.069974,0,20576,20576,0.0186254
3,508,44.5889,0.0,0.0,0.069574,0,20576,20576,0
4,509,44.0311,0.0,1.0,0.070049,0,20576,20576,0
...,...,...,...,...,...,...,...,...,...
247,752,71.0852,1.0,1.0,0.053183,0,47577.2,47577.2,0
248,753,70.1616,1.0,1.0,0.053001,0,47577.2,47577.2,0
249,754,71.4151,1.0,1.0,0.053152,0,47577.2,47577.2,0
250,755,72.0088,1.0,1.0,0.053664,660,20.53,47515.4,-0.00129852


In [27]:
close['SH600150']['pnl'].mean()

0.00036151575115725334

In [28]:
close['SH600150']['pnl'].std()

0.0620485377299377

In [29]:
pnl = []

for x in stock:
    for i in range(0,len(close[x])):
        pnl.append(close[x].iloc[i,8])

In [30]:
sharpe_ratio = np.sqrt(252) * np.mean(pnl) / np.std(pnl)
print('Sharpe Ratio :', round(sharpe_ratio,4))

Sharpe Ratio : 0.3053


# Metric 3: Number of days making profit

In [31]:
for x in stock:
    close[x].set_index('day', inplace = True)

In [32]:
close['SH600150']

,Close,label5_preds,label10_pred,LR_pred,stock owned,cash owned,total asset,pnl
day,,,,,,,,
505,43.5061,1.0,1.0,0.070575,459,17.72,19974,0
506,43.9983,0.0,1.0,0.070350,459,17.72,20199.8,0.0113032
507,44.8185,0.0,0.0,0.069974,0,20576,20576,0.0186254
508,44.5889,0.0,0.0,0.069574,0,20576,20576,0
509,44.0311,0.0,1.0,0.070049,0,20576,20576,0
...,...,...,...,...,...,...,...,...
752,71.0852,1.0,1.0,0.053183,0,47577.2,47577.2,0
753,70.1616,1.0,1.0,0.053001,0,47577.2,47577.2,0
754,71.4151,1.0,1.0,0.053152,0,47577.2,47577.2,0


In [33]:
close['SH600150'].loc[505, :] 

Close             43.5061
label5_preds            1
label10_pred            1
LR_pred         0.0705747
stock owned           459
cash owned          17.72
total asset         19974
pnl                     0
Name: 505, dtype: object

In [34]:
df = pd.DataFrame({'day' : range(505,757)}) 
df.set_index('day', inplace = True)
for x in stock:
    df[x] = ""
    for i in range(505,757):
        df.loc[i, x] = close[x].loc[i,'pnl']
df.loc[:,'pnl_total'] = df.sum(axis=1)
df["profit"] = np.where(df["pnl_total"]>0,1,0)

In [35]:
df.head()

,SH600000,SH600010,SH600015,SH600016,SH600018,SH600028,SH600030,SH600036,SH600048,SH600050,...,SH601766,SH601800,SH601818,SH601857,SH601901,SH601988,SH601989,SH601998,pnl_total,profit
day,,,,,,,,,,,,,,,,,,,,,
505,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000000,0
506,0,0,0,0,0,0,0.0143871,0,0,0,...,0,0,0,0,0,0,0,0,0.035556,1
507,0,0,0,0,0,0,0.0336841,0,0,0,...,0,0,0,0,0,0,0,0,0.031949,1
508,0,0,0,0,0,0,0.00914727,0,0,0,...,0,0,0,0,0,0,0,0,-0.018332,0
509,0,0,0,0,0,0,-0.00849754,0,0,0,...,0,0,0,0,0,0,0,0,-0.007130,0


In [36]:
profit_days = df['profit'].sum()
total_days = df['profit'].count()

print(f'There are {profit_days} days out of {total_days} days with profit.')

There are 142 days out of 252 days with profit.
